# Data Sourcing

This notebook will make API call and source data.

* S&P 500
   * P/E ratio for S&P (as a proxy for investors' emotional attitude towards the market
* Indian market's version of S&P and Dow Jones
* Market indices based on varying investment philosophies in both countries.


#### Ticker Symbols

* S & P 500 - VOO (Vanguard S&P 500 ETF(Ultra low-cost, strong tracking)
* JP Morgan Equity Premium Income ETF (Actively managed) - JEPI


In [60]:
import pandas as pd, requests

In [61]:
apiKey = 'RQRA0RQ6ZZAVMA26'

In [135]:
r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SPy&outputsize=full&apikey={apiKey}")

In [136]:
data = r.json()
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'SPy',
  '3. Last Refreshed': '2025-07-23',
  '4. Output Size': 'Full size',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2025-07-23': {'1. open': '631.7500',
   '2. high': '634.2000',
   '3. low': '629.7300',
   '4. close': '634.1900',
   '5. volume': '70310677'},
  '2025-07-22': {'1. open': '629.1000',
   '2. high': '629.7300',
   '3. low': '626.1900',
   '4. close': '628.8600',
   '5. volume': '60046288'},
  '2025-07-21': {'1. open': '628.7700',
   '2. high': '631.5400',
   '3. low': '628.3400',
   '4. close': '628.7700',
   '5. volume': '63374984'},
  '2025-07-18': {'1. open': '629.3000',
   '2. high': '629.4700',
   '3. low': '626.4600',
   '4. close': '627.5800',
   '5. volume': '65621645'},
  '2025-07-17': {'1. open': '624.4000',
   '2. high': '628.4048',
   '3. low': '624.1800',
   '4. close': '628.0400',
   '5. volume': '68885683'},
  '2025-07-16': {'1. open': '62

{}

In [57]:
ETF = requests.get(f"https://www.alphavantage.co/query?function=ETF_PROFILE&symbol=SPG&outputsize=full&apikey={apiKey}")

In [58]:
ETFdata = ETF.json()

In [59]:
ETFdata

{}